In [ ]:
import sys, glob, os
import h5py
import math
import csv
import numpy as np
import pandas as pd
import statistics as stat
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from textwrap import wrap

#from ml4c3.tensormap.TensorMap import update_tmaps
# matplotlib widget

In [ ]:
#data_dir = '/media/ml4c3/hd5_cov/'
data_dir = '/home/ridwan/temp_hd5_data/'
all_files = glob.glob(os.path.join(data_dir,"*.hd5"))
len(all_files)

In [ ]:
#vent_alarms = ['r_vent_alarm_peep_min', 'r_vent_alarm_press_max', 'r_vent_alarm_rr_max', 'r_vent_alarm_rr_min', 'r_vent_alarm_spo2_max']
vent_sigs = ['r_vent_exp_tidal_volume', 'r_vent_map', 'r_vent_minute_ventilation',\
             'r_vent_peep','r_vent_plateau_pressure', 'r_vent_tidal_volume_set',\
             'r_vent_press_supp_set', 'r_vent_resp_rate_(set)', 'r_vent_pip_set','r_ventilator_on']

In [ ]:
mode = 'edw'

levels = ['flowsheet']

vent_sig_names = {levels[0]:set(vent_sigs)}

In [ ]:
plt.rcParams['figure.figsize'] = [10, 30]

In [ ]:
k = 0

for fpath in all_files:
    
    k = k + 1
    # get mrn from filepath
    mrn = fpath.split('/')[-1]
    mrn = mrn.split('.')[0]
    print( k, len(all_files))
    
    # read HD5 file
    hd5file = h5py.File(fpath, 'r')
    
    file_modes = list(hd5file.keys())    
    # mode = 'edw'
    # for mode in file_modes:
    if mode in file_modes:
    
        # how many csn for this file
        csns = list(hd5file[mode].keys())
        for csn in csns:
            # print ('csn: '+ csn)
            # mrn_csn_list.append((mrn,csn))
            
            level_sig_dur_hours = {}
            
            file_levels = list(hd5file[mode][csn].keys())
            # levels = ['flowsheet']
            for level in levels:
                
                if level in file_levels:
                    group_name = '/'+ mode +'/'+ csn +'/'+ level
                    # print ('level: '+ group_name)
                    
                    sig_names = list(set(hd5file[group_name].keys()) & set(vent_sig_names[level]))
                    
                    if len(sig_names)>0:
                        
                        sig_groups = [hd5file.get(group_name +'/'+ sig_name) for sig_name in sig_names]
                        
                        figfile_name = '/home/ridwan/myDrafts/figs_temp/' + mrn + '_' +csn + '_' +level +'_r_sig.png'
                        fig,axs = plt.subplots(len(sig_names),1,sharex='all',squeeze=False)
                        t = 0
                        # all_sig_names[level].update(sig_list)
                        xlim_min = datetime.now()
                        xlim_max = datetime.utcfromtimestamp(1)
                        
                        for t in range(len(sig_names)):
                            sig_group = sig_groups[t]
                            sig_time = list(sig_group.get('time'))
                            sig_value = list(sig_group.get('value'))
                            sig_time_dttm = [datetime.utcfromtimestamp(temp_time) for temp_time in sig_time]
                            
                            xlim_min = min(xlim_min,min(sig_time_dttm))
                            xlim_max = max(xlim_max,max(sig_time_dttm))
                            
                            axs[t,0].plot(sig_time_dttm,sig_value,'.')
                            axs[t,0].set_xlim(xlim_min,xlim_max)
                            axs[t,0].xaxis.set_major_locator(ticker.MultipleLocator(2))
                            axs[t,0].set_ylabel('\n'.join(wrap(sig_names[t], 10)))
                        fig.autofmt_xdate()
                        plt.savefig(figfile_name)
                        plt.close()
                        
                    else:
                        print('missed sig:' + mrn + '/' + group_name)
                        
                        
##